# TODO

* Write method to present data
* 

In [15]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")

/Users/frederikwarburg/anaconda3/envs/neuralcoref/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [13]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize

In [14]:
data = pd.read_csv('data/gendered-pronoun-resolution/test_stage_1.tsv', sep = '\t')
data.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


In [49]:
text = data['Text'][0]


In [54]:
#text = 'This is the first sentence. This is the second sentence.'
prediction = predictor.predict(
  document=text
)

In [55]:
[key for key in prediction]

['top_spans', 'predicted_antecedents', 'document', 'clusters']

In [72]:
def check_in_top_span(i, top_spans):
    return_values = False, -1
    span = []
    for num, top_span in enumerate(top_spans):
        
        if i >= top_span[0] and i <= top_span[1]:
            if return_values[0]:
                # If sequence are overlapping, we only show the longest
                if span[1] - span[0] < top_span[1] - top_span[0]:
                    span = top_span
                    return_values = True, num
            else:
                span = top_span
                return_values = True, num
        
    return return_values

def show_top_spans(prediction, text):
    tokens = word_tokenize(text)

    std_out = ''
    prev_top_span = -1
    colors = ['red', 'blue','green', 'cyan', 'yellow']
    for i, token in enumerate(tokens):
                                   
        in_top_span, num = check_in_top_span(i, prediction['top_spans'])
        if in_top_span:
            if prev_top_span == num:
                std_out += colored(' ', colors[num%len(colors)], attrs=['reverse', 'blink'])
            else:
                std_out += ' '
            std_out += colored(token, colors[num%len(colors)], attrs=['reverse', 'blink'])
            prev_top_span = num
        else:
            std_out += ' '
            std_out += token
            prev_top_span = -1

    print(std_out)
    
show_top_spans(prediction, text)

 Zoe Telford -- played the police officer girlfriend of Simon , Maggie . Dumped by Simon in the final episode of series 1 , after he slept with Jenny , and is not seen again . Phoebe Thomas played Cheryl Cassidy , Pauline 's friend and also a year 11 pupil in Simon 's class . Dumped her boyfriend following Simon 's advice after he would n't have sex with her but later realised this was due to him catching crabs off her friend Pauline .


In [73]:
def check_in_cluster(i, clusters):
    
    for c, cluster in enumerate(clusters):
        for span in cluster:
            if i >= span[0] and i <= span[1]:
                return True, c
    
    return False, -1
        

def show_predicted_clusters(prediction, text):
    tokens = word_tokenize(text)

    std_out = ''
    colors = ['red', 'blue','green', 'cyan', 'yellow','black']
    prev_cluster  =-1
    for i, token in enumerate(tokens):

        in_predicted_cluster, num = check_in_cluster(i, prediction['clusters'])
        if in_predicted_cluster:
            if prev_cluster == num:
                std_out += colored(' ', colors[num%len(colors)], attrs=['reverse', 'blink'])
            else: 
                std_out += ' '
            std_out += colored(token, colors[num%len(colors)], attrs=['reverse', 'blink'])
            prev_cluster = num
        else:
            std_out += ' '
            std_out += token
            prev_cluster = -1

    print(std_out)
    
show_predicted_clusters(prediction, text)

 Zoe Telford -- played the police officer girlfriend of Simon , Maggie . Dumped by Simon in the final episode of series 1 , after he slept with Jenny , and is not seen again . Phoebe Thomas played Cheryl Cassidy , Pauline 's friend and also a year 11 pupil in Simon 's class . Dumped her boyfriend following Simon 's advice after he would n't have sex with her but later realised this was due to him catching crabs off her friend Pauline .
